# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,olonkinbyen,70.9221,-8.7187,5.43,89,83,3.35,SJ,1722715805
1,gamba,-2.6500,10.0000,22.42,74,71,4.22,GA,1722715806
2,albany,42.6001,-73.9662,28.43,66,86,2.16,US,1722715656
3,thompson,55.7435,-97.8558,21.90,26,66,4.83,CA,1722715808
4,jamestown,42.0970,-79.2353,25.77,83,100,2.06,US,1722715612


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_data_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Lng", "Lat", "City", "Humidity"]
)

# Display the map
city_data_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
conditions_df = city_data_df[(city_data_df["Max Temp"] >= 5) & (city_data_df["Max Temp"] <= 40) &
                            (city_data_df["Humidity"] >= 20) & (city_data_df["Humidity"] <=90)]
# Drop any rows with null values
Ideal = conditions_df.dropna()

# Display sample data
Ideal

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,olonkinbyen,70.9221,-8.7187,5.43,89,83,3.35,SJ,1722715805
1,gamba,-2.6500,10.0000,22.42,74,71,4.22,GA,1722715806
2,albany,42.6001,-73.9662,28.43,66,86,2.16,US,1722715656
3,thompson,55.7435,-97.8558,21.90,26,66,4.83,CA,1722715808
4,jamestown,42.0970,-79.2353,25.77,83,100,2.06,US,1722715612
...,...,...,...,...,...,...,...,...,...
563,jaisalmer,26.9147,70.9181,26.58,88,100,3.64,IN,1722716871
564,luganville,-15.5333,167.1667,23.87,85,100,5.16,VU,1722716873
565,mocajuba,-2.5842,-49.5072,35.10,26,42,1.76,BR,1722716874


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal[[
                  'City',
                  'Country',
                  'Lat',
                  'Lng',
                  'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
0,olonkinbyen,SJ,70.9221,-8.7187,89,
1,gamba,GA,-2.6500,10.0000,74,
2,albany,US,42.6001,-73.9662,66,
3,thompson,CA,55.7435,-97.8558,26,
4,jamestown,US,42.0970,-79.2353,83,
...,...,...,...,...,...,...
563,jaisalmer,IN,26.9147,70.9181,88,
564,luganville,VU,-15.5333,167.1667,85,
565,mocajuba,BR,-2.5842,-49.5072,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
#Latitude = ?
#Longitude = ?
categories="accommodation.hotel"
limit = 10
#conditions=
#filters= f"circle:{longitude}{latitude}"
#bias = f"proximity:{longitude}{latitude}"
radius = 10000
params = {
    "categories": categories,
    "limit":limit,
    "apiKey": geoapify_key
}

#    "filter": filters,"bias": bias,

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    #latitude = hotel_df.loc[index, "Lat"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    #longitude = hotel_df.loc[index, "Lng"]
    


    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
olonkinbyen - nearest hotel: No hotel found
gamba - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
jamestown - nearest hotel: DoubleTree Jamestown
barah - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
brigantine - nearest hotel: La Sammana
blackmans bay - nearest hotel: Villa Howden
tazovsky - nearest hotel: Тазовчанка
remire-montjoly - nearest hotel: Complexe Belova
ta`u - nearest hotel: No hotel found
qamdo - nearest hotel: 银杏商务宾馆
vorkuta - nearest hotel: "Мегаполис"
bethel - nearest hotel: Hampton Inn Danbury
port shepstone - nearest hotel: The Spot Backpackers'
atafu village - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
bikenibeu village - nearest hotel: Otintaai Hotel
utrik - nearest hotel: No hotel found
mokshan - nearest hotel: Родник
cabo san lucas - nearest hotel: Comfort Rooms
bilibino - nearest hotel: No hotel found
l

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
0,olonkinbyen,SJ,70.9221,-8.7187,89,No hotel found
1,gamba,GA,-2.6500,10.0000,74,No hotel found
2,albany,US,42.6001,-73.9662,66,No hotel found
3,thompson,CA,55.7435,-97.8558,26,Thompson Inn
4,jamestown,US,42.0970,-79.2353,83,DoubleTree Jamestown


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_data_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Lng", "Lat", "City", "Humidity","Hotel Name", "Country"]
)

# Display the map
hotel_data_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)